# **Written by Yoonjin Oh**

- Code Reference: https://github.com/databrickslabs/dolly/blob/master/training/trainer.py)
- https://drive.google.com/file/d/1hzaf-5vUfYWmhWdqSwq6BgRoj0SlEDgs/view?usp=sharing
- 허깅페이스의 transformers 라이브러리 설명 위키독스: https://wikidocs.net/166799
- transformers 라이브러리 내 Data Collator 관련 독스: https://huggingface.co/docs/transformers/v4.14.1/main_classes/data_collator


[ 할 일 ]
1. 마지막 단계에서 WandB 추가 필요
1.5. 코드 사이사이 프린트 함수 추가 필요
2. Collator 관련 함수 추가
3. Val - Test 변수명 통일 (완료)

## **0. Configuration**

In [ ]:
!pip install torch
!pip install transformers

In [ ]:
!pip install datasets

In [62]:
# 기본 라이브러리 (클래스)

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, sampler, random_split

import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

In [4]:
# 출처 dolly/trainig/trainer.py

# import logging
from datasets import Dataset, load_dataset
from functools import partial                # 하나 이상의 인수가 이미 채워진 함수의 새 버전을 만들기 위해 사용
from transformers import (
    AutoModelForCausalLM,                    # (확률기반으로 다음 단어를예측하는) 사전학습 모델 임포트 위한 클래스
    AutoTokenizer,                           # 토크나이저 임포트 위한 클래스
    # DataCollatorForLanguageModeling,       # 지정한 크기의 배치로 데이터 샘플을 구성하는 클래스 (배치 단위로 패딩 적용 가능)
    PreTrainedTokenizer,                     # 사전학습 된 모델에 맞는 토크나이저 (만약 주어진 데이터셋에 대해 동작하지 않을 경우, 직접 처음부터 토크나이저 구현 필요)
    # Trainer,                                 # 실제 학습에 필요한 클래스
    # TrainingArguments,
    set_seed,
)

In [5]:
# 데이터셋 구성 확인 ('train'만 존재하는지 여부 - 확인 결과 train 만 존재한다.)

from datasets import get_dataset_split_names
get_dataset_split_names("databricks/databricks-dolly-15k")

['train']

In [6]:
# 데이터셋 크기 확인

dataset_org = load_dataset("databricks/databricks-dolly-15k")["train"]
dataset_org_size = len(dataset_org)
dataset_org_size

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

15011

In [7]:
# 데이터셋 분할 (train:test = 8:2)

train_size = int(dataset_org_size * 0.8)
test_size = dataset_org_size - train_size
train_dataset, test_dataset = random_split(dataset_org, [train_size, test_size])

# 분할 결과 (각 사이즈) 확인

print(f"Training Data Size : {len(train_dataset)}")
print(f"Test Data Size : {len(test_dataset)}")

Training Data Size : 12008
Test Data Size : 3003


In [37]:
# 상수 선언 (각각의 의미에 대해 정확히 이해한 후 수정 혹은 삭제 필요)

INTRO_BLURB = (
    "Below is an instruction that describes a task. Write a response that appropriately completes the request."
)
INSTRUCTION_KEY = "### Instruction:"
INPUT_KEY = "Input:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"
RESPONSE_KEY_NL = f"{RESPONSE_KEY}\n"

DEFAULT_SEED = 42
DEFAULT_TRAINING_DATASET = train_dataset
DEFAULT_TEST_DATASET = test_dataset # 추가


# This is a training prompt that does not contain an input string.  The instruction by itself has enough information
# to respond.  For example, the instruction might ask for the year a historic figure was born.
PROMPT_NO_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{response_key}
{response}

{end_key}""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    response_key=RESPONSE_KEY,
    response="{response}",
    end_key=END_KEY,
)



# This is a training prompt that contains an input string that serves as context for the instruction.  For example,
# the input might be a passage from Wikipedia and the intruction is to extract some information from it.

PROMPT_WITH_INPUT_FORMAT = """{intro}

{instruction_key}
{instruction}

{input_key}
{input}

{response_key}
{response}

{end_key}""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction="{instruction}",
    input_key=INPUT_KEY,
    input="{input}",
    response_key=RESPONSE_KEY,
    response="{response}",
    end_key=END_KEY,
)

# **1. Dataset Import**


In [68]:
# 데이터셋 로드 함수

def load_dataset(dataset):

    # 각 인스턴스(record) 마다의 instruction / response / context 확인 (= 원본 데이터셋의 column 이름)
    instructions = []
    responses = []
    contexts = []

    def _add_text(rec):
      instruction = rec["instruction"]
      response = rec["response"]
      context = rec.get("context")

      if not instruction:
        raise ValueError(f"Expected an instruction in: {rec}")

      if not response:
        raise ValueError(f"Expected a response in: {rec}")

      # 새로운 'text' 필드(열) 추가 (추후 PROMPT_WITH/NO_INPUT_FORMAT 상수 정의 필요)
      if context:
        rec["text"] = PROMPT_WITH_INPUT_FORMAT.format(instruction=instruction, response=response, input=context)

      # context 에 대한 정보가 없는 열(instance)도 존재
      else:
        rec["text"] = PROMPT_NO_INPUT_FORMAT.format(instruction=instruction, response=response)

      return rec


    # 데이터셋의 모든 인스턴스에 _add_text 함수 적용
    if not isinstance(dataset, Dataset):
        dataset = Dataset.from_pandas(dataset)  # Convert to datasets.Dataset

    dataset = dataset.map(_add_text)

    return dataset

# **2. Tokenizer Import**

In [39]:
# 토크나이저 로드 함수 (문제 2 - Special Token)

def load_tokenizer(pretrained_model_name):
  # logger.info(f"Loading tokenizer for {pretrained_model_name}")
  tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
  tokenizer.pad_token = tokenizer.eos_token   # 패딩 토큰으로 EOS(End of Sequence) 토큰 지정
  tokenizer.add_special_tokens({"additional_special_tokens": [END_KEY, INSTRUCTION_KEY, RESPONSE_KEY_NL]})

  '''
  토큰화 혹은 모델 훈련 과정에서 특정 기능을 수행하는 Special Tokens 추가

  END_KEY: input 의 끝(end)을 표시하는 토큰 (instruction 과 response 와 같이 input 안에서 완전히 다른 부분을 분리하는 역할)
  INSTRUCTION_KEY: input 내에서 instruction 부분을 표시하는 토큰
  RESPONSE_KEY_NL: input 내에서 response 부분을 표시하는 토큰, 이때 'NL' 은 New Line 을 의미한다.

  '''

  return tokenizer


# **3. Model Import**

In [40]:
# 모델 로드 함수

def load_model(pretrained_model_name):
  model = AutoModelForCausalLM.from_pretrained(
      pretrained_model_name, trust_remote_code=True
      # trust_remote_code : 허깅페이스에서 모델 로드할 때 유용
      # use_cache: 웨이트를 캐시에 저장할 것인지 여부
  )
  return model


In [42]:
# 모델과 그에 대응하는 토크나이저를 한번에 튜플로 로드하는 함수

def get_model_tokenizer(
    pretrained_model_name
):
  tokenizer = load_tokenizer(pretrained_model_name)
  model = load_model(pretrained_model_name)

  # 토크나이저의 vocab 사이즈에 맞게 모델의 토큰 임베딩 사이즈 조정 (default로 토크나이저와 모델의 vocab 사이즈는 다를 것)
  model.resize_token_embeddings(len(tokenizer))

  return model, tokenizer


# **4. Dataset Preprocessing (알 수 없음)**

*   closed_qa
*   open_qa
*   information_extraction
*   summarization
...

**해야할 일: 각 모델의 Training Input Format 에 맞게, 주어진 데이터를 전처리하여 이를 학습시키는 것이 목표?**

**Try 1. for General Training (NO Task-specific)**

In [43]:
# 배치(Batch) 단위로 전처리 (토크나이징)

def preprocess_batch(batch, tokenizer, max_length) -> dict:
  return tokenizer(
      batch["text"],
      max_length=max_length,
      truncation=True,
      # padding=True,
      # 모델에 따라 필요 시, max_length 인자 추가
  )

In [44]:
# 데이터셋 전처리

def preprocess_dataset(tokenizer, max_length, seed, on_dataset):

  # (_add_text("text" col) 및 special token의 정체를 밝혀내야 한다. 그렇지 않으면, 실질적으로 load_training dataset 함수의 존재 이유가 없다.)
  dataset = load_dataset(on_dataset)
  _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)

  encodings = dataset.map(
    _preprocessing_function,
    batched = True,                                                              # 각각의 인스턴스가 아닌 배치 단위로 함수 적용
    remove_columns=["instruction", "context", "response", "text", "category"],   # 필요 시 제거
  )

  encodings = encodings.filter(lambda rec: len(rec["input_ids"]) < max_length)  # max_length 보다 한 인스턴스의 input_ids 길이가 짧아야 한다. (trunication 으로 정보가 소실된 인스턴스 필터링하는 역할)
  encodings = encodings.shuffle(seed=seed)

  return encodings


In [45]:
# Dataset 클래스 생성

class DollyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [46]:
# DataLoader 로 저장

def to_dataloader(encodings, batch_size): # 메모리 이슈로 인해 배치 사이즈 4
  dataset = DollyDataset(encodings)
  dataloader = DataLoader(dataset, batch_size, shuffle=True)

# **5. Train and Evaluate**

In [60]:
def train_and_evaluate(
    *,
    input_model: str,
    epochs: int,
    lr: float,
    batch_size: int,
    seed: int,
    training_dataset: str = DEFAULT_TRAINING_DATASET,
    test_dataset: str = DEFAULT_TEST_DATASET,
):

    # 추후 동일한 결과 복제 목적
    set_seed(seed)


    ### (1) 모델 및 토크나이저 임포트 ###
    model, tokenizer = get_model_tokenizer(
        pretrained_model_name=input_model
    )
    model.to(device)
    opimizer = AdamW(model.parameters(), lr)


    ### (2) 기본 세팅 ###
    conf = model.config
    max_length = None

    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
      max_length = getattr(model.config, length_setting, None)
      if max_length:
        break
    if not max_length:
      max_length = 1024 # 필요시 (모델 메모리 사용량이 너무 클 때, 수정)

    # (총 훈련 시간 기록 목적)
    whole_train_eval_time = time.time()

    # (Loss 값 기록 목적)
    train_losses = []
    test_losses = []

    # Train Loss 값 출력 주기 지정
    print_every = 1000
    print('using device:', device)


    ### (3) 데이터 전처리 ###
    processed_dataset = preprocess_dataset(tokenizer=tokenizer, max_length=max_length, seed=seed, on_dataset=training_dataset) # train
    train_loader = to_dataloader(processed_dataset, batch_size)
    processed_dataset = preprocess_dataset(tokenizer=tokenizer, max_length=max_length, seed=seed, on_dataset=test_dataset) # test
    test_loader = to_dataloader(processed_dataset, batch_size)


    ### (4) 훈련 ###
    for epoch in range(epochs):
      epoch_time = time.time()

      # train 모드로 설정
      model.train()

      loss_of_epoch = 0

      print("############Train############")

      for batch_idx,batch in enumerate(train_loader):

        optim.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        # 모델에 따라서 수정 필요
        outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
        loss = outputs[0]

        # do a backwards pass
        loss.backward()

        # update the weights
        optim.step()

        # Find the total loss
        loss_of_epoch += loss.item()

        if (batch_idx+1) % print_every == 0:
          print("Batch {:} / {:}".format(batch_idx+1,len(train_loader)),"\nLoss:", round(loss.item(),1),"\n")

      loss_of_epoch /= len(train_loader)
      train_losses.append(loss_of_epoch)


      ### (5) 평가 ###
      # 평가모드로 설정
      model.eval()

      print("############Evaluate############")

      loss_of_epoch = 0

      for batch_idx,batch in enumerate(test_loader):

        with torch.no_grad():

          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          start_positions = batch['start_positions'].to(device)
          end_positions = batch['end_positions'].to(device)

          outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
          loss = outputs[0]

          # 전체 (총) Loss 값
          loss_of_epoch += loss.item()

        if (batch_idx+1) % print_every == 0:
          print("Batch {:} / {:}".format(batch_idx+1,len(test_loader)),"\nLoss:", round(loss.item(),1),"\n")

      loss_of_epoch /= len(test_loader)
      test_losses.append(loss_of_epoch)


      # 매 에포크마다의 소요 시간 및 train/test Loss 출력
      print("\n-------Epoch ", epoch+1,
            "-------"
            "\nTraining Loss:", train_losses[-1],
            "\nValidation Loss:", test_losses[-1],
            "\nTime: ",(time.time() - epoch_time),
            "\n-----------------------",
            "\n\n")


    # 전체 훈련 및 평가 시간 출력
    print("Total training and evaluation time: ", (time.time() - whole_train_eval_time))


    ### (6) 모델 저장 ###
    torch.save(model,"/content/drive/MyDrive/Experiment") # VSCODE 에서 실행 시 수정 필요

    ### (7) Loss 그래프 그리기 ###
    fig,ax = plt.subplots(1,1,figsize=(15,10))

    ax.set_title("Train and Validation Losses",size=20)
    ax.set_ylabel('Loss', fontsize = 20)
    ax.set_xlabel('Epochs', fontsize = 25)
    _=ax.plot(train_losses)
    _=ax.plot(test_losses)
    _=ax.legend(('Train','Val'),loc='upper right')


# **5.5 Compute Metric (Accuracy)**

# **6. Run**

In [48]:
# GPU(디바이스) 설정
USE_GPU = True
dtype = torch.float32 # 오류 시 추후 수정

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [58]:
# 모델 설정
from transformers import GPT2Config, GPT2LMHeadModel, AdamW
input_model = "gpt2"

# 하이퍼파라미터 설정
epochs = 3
lr = 5e-5
batch_size = 4
seed = DEFAULT_SEED

In [69]:
# 실행
train_and_evaluate(
    input_model=input_model,
    epochs=epochs,
    lr=lr,
    batch_size=batch_size,
    seed=seed,
)

using device: cuda


AttributeError: ignored

# **참고/폐기 코드**

In [ ]:
model = T5ForQuestionAnswering.from_pretrained('t5-base').to(device)
optim = AdamW(model.parameters(), lr=5e-5)  # Adam with Weight Decay

epochs = 3

In [ ]:
whole_train_eval_time = time.time()

train_losses = []
val_losses = []

# Train Loss 값 출력 주기 지정
print_every = 100
print('using device:', device)

# Train
for epoch in range(epochs):
  epoch_time = time.time()

  # Set model in train mode
  model.train()

  loss_of_epoch = 0

  print("############Train############")

  for batch_idx,batch in enumerate(train_loader):

    optim.zero_grad()

    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    start_positions = batch['start_positions'].to(device)
    end_positions = batch['end_positions'].to(device)

    outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
    loss = outputs[0]
    # do a backwards pass
    loss.backward()
    # update the weights
    optim.step()
    # Find the total loss
    loss_of_epoch += loss.item()

    if (batch_idx+1) % print_every == 0:
      print("Batch {:} / {:}".format(batch_idx+1,len(train_loader)),"\nLoss:", round(loss.item(),1),"\n")

  loss_of_epoch /= len(train_loader)
  train_losses.append(loss_of_epoch)

  ##########Evaluation##################

  # Set model in evaluation mode
  model.eval()

  print("############Evaluate############")

  loss_of_epoch = 0

  for batch_idx,batch in enumerate(val_loader):

    with torch.no_grad():

      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      start_positions = batch['start_positions'].to(device)
      end_positions = batch['end_positions'].to(device)

      outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
      loss = outputs[0]
      # Find the total loss
      loss_of_epoch += loss.item()

    if (batch_idx+1) % print_every == 0:
       print("Batch {:} / {:}".format(batch_idx+1,len(val_loader)),"\nLoss:", round(loss.item(),1),"\n")

  loss_of_epoch /= len(val_loader)
  val_losses.append(loss_of_epoch)

  # Print each epoch's time and train/val loss
  print("\n-------Epoch ", epoch+1,
        "-------"
        "\nTraining Loss:", train_losses[-1],
        "\nValidation Loss:", val_losses[-1],
        "\nTime: ",(time.time() - epoch_time),
        "\n-----------------------",
        "\n\n")

print("Total training and evaluation time: ", (time.time() - whole_train_eval_time))